In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

shared_dir = '/Users/prateek/Desktop/ML/Major/data/'
input_shape = [32, 32]
batch = 32

training_set = image_dataset_from_directory(shared_dir,
                                            validation_split=0.2,
                                            subset="training",
                                            batch_size=batch,
                                            seed=1,
                                            color_mode='rgb',
                                            image_size=input_shape)

val_dataset = image_dataset_from_directory(shared_dir,
                                            validation_split=0.2,
                                            subset="validation",
                                            batch_size=batch,
                                            seed=1,
                                            color_mode='rgb',
                                            image_size=input_shape)

input_shape.append(3)
print(input_shape)

Found 27558 files belonging to 2 classes.
Using 22047 files for training.
Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-03-16 17:05:35.333973: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-16 17:05:35.334082: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 27558 files belonging to 2 classes.
Using 5511 files for validation.


In [2]:
import keras

data_augmentation = keras.Sequential(
    [       tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
            tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    ]
)


In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D
from keras import Model, Input
from keras.applications.xception import Xception, preprocess_input

epochs = 300

base_model = Conv2D(4, kernel_size=(24, 24),activation='relu',input_shape=(150, 150, 3))

base_model.trainable = False


inputs = Input(shape=(150, 150, 3))
x = data_augmentation(inputs) 
x = preprocess_input(inputs)
x = base_model(x, training=False)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
outputs = Dense(1)(x)
model = Model(inputs, outputs)
print(model.summary())
model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=True),metrics=keras.metrics.BinaryAccuracy())

model.fit(training_set, epochs=epochs, validation_data=val_dataset)

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 tf.math.truediv_2 (TFOpLamb  (None, 150, 150, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_2 (TFOpLam  (None, 150, 150, 3)      0         
 bda)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 127, 127, 4)       6916      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 63, 63, 4)        0         
 2D)                                                             
                                                           

2022-03-16 17:08:31.251382: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


689/689 [==============================] - ETA: 0s - loss: 0.6715 - binary_accuracy: 0.6241

2022-03-16 17:08:48.316967: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


689/689 [==============================] - 21s 30ms/step - loss: 0.6715 - binary_accuracy: 0.6241 - val_loss: 0.6433 - val_binary_accuracy: 0.6110
Epoch 2/50
689/689 [==============================] - 19s 28ms/step - loss: 0.6451 - binary_accuracy: 0.6523 - val_loss: 0.6476 - val_binary_accuracy: 0.6692
Epoch 3/50
689/689 [==============================] - 20s 28ms/step - loss: 0.6240 - binary_accuracy: 0.6637 - val_loss: 0.6430 - val_binary_accuracy: 0.6483
Epoch 4/50
689/689 [==============================] - 19s 28ms/step - loss: 0.6002 - binary_accuracy: 0.6765 - val_loss: 0.6957 - val_binary_accuracy: 0.6093
Epoch 5/50
689/689 [==============================] - 19s 28ms/step - loss: 0.5942 - binary_accuracy: 0.6837 - val_loss: 0.6313 - val_binary_accuracy: 0.6567
Epoch 6/50
689/689 [==============================] - 20s 28ms/step - loss: 0.5932 - binary_accuracy: 0.6878 - val_loss: 0.6928 - val_binary_accuracy: 0.6654
Epoch 7/50
689/689 [==============================] - 20s 29ms/

In [6]:
base_model.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),  
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=tf.keras.metrics.BinaryAccuracy())
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# rm -rf logs
%load_ext tensorboard
log_folder = 'logs'
callbacks = [
            EarlyStopping(patience = 5),
            TensorBoard(log_dir=log_folder)
            ]

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
# rm -rf logs
%load_ext tensorboard
log_folder = 'logs'
callbacks = [
            EarlyStopping(patience = 5),
            TensorBoard(log_dir=log_folder)
            ]
model.fit(training_set, epochs=epochs,validation_data=val_dataset,callbacks=callbacks)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Epoch 1/15


2022-03-16 19:20:10.474866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


689/689 [==============================] - ETA: 0s - loss: 0.4162 - binary_accuracy: 0.7975

2022-03-16 19:21:15.427303: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


689/689 [==============================] - 69s 99ms/step - loss: 0.4162 - binary_accuracy: 0.7975 - val_loss: 0.6957 - val_binary_accuracy: 0.6668
Epoch 2/15
689/689 [==============================] - 68s 99ms/step - loss: 0.4093 - binary_accuracy: 0.8022 - val_loss: 0.6876 - val_binary_accuracy: 0.6832
Epoch 3/15
689/689 [==============================] - 68s 98ms/step - loss: 0.4045 - binary_accuracy: 0.8068 - val_loss: 0.6848 - val_binary_accuracy: 0.6794
Epoch 4/15
689/689 [==============================] - 72s 104ms/step - loss: 0.4031 - binary_accuracy: 0.8057 - val_loss: 0.6871 - val_binary_accuracy: 0.6837
Epoch 5/15
689/689 [==============================] - 73s 106ms/step - loss: 0.3991 - binary_accuracy: 0.8098 - val_loss: 0.6784 - val_binary_accuracy: 0.6861
Epoch 6/15
689/689 [==============================] - 73s 107ms/step - loss: 0.3963 - binary_accuracy: 0.8130 - val_loss: 0.6745 - val_binary_accuracy: 0.6839
Epoch 7/15
689/689 [==============================] - 163s 2